# Evaluate Data Quality Issues in 'brands.json'

In [1]:
#Importing Libraries
import pandas as pd
import json
import numpy as np

In [2]:
df_brands = pd.read_json('original_data/brands.json',lines=True)
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [3]:
df_brands.shape

(1167, 8)

### Lets understand the data type for each column

In [4]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


### We have 6 columns with type as object, checking  how the first row looks

In [5]:
for i in df_brands.columns:
    if df_brands[i].dtype == 'object':
        print(f"{i}: {df_brands[i][0]}\n")

_id: {'$oid': '601ac115be37ce2ead437551'}

category: Baking

categoryCode: BAKING

cpg: {'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}

name: test brand @1612366101024

brandCode: nan



### Correcting format for '_id' column and changing the name of '_id' to 'user_id'

In [6]:
df_brands['_id'] = df_brands['_id'].apply(lambda x: x['$oid'])
df_brands.rename(columns={'_id': 'brand_id'}, inplace=True)

### Correcting format for 'cpg' column, it is a dictionary with two keys, so we can make these keys 2 different columns for brands table - cpg_id and cpg_ref

In [7]:
def extract_ref_and_id(row):
    ref = row.get('$ref', None)
    oid = row.get('$id', {}).get('$oid', None)
    return ref, oid
df_brands['cpg_ref'], df_brands['cpg_id'] = zip(*df_brands['cpg'].apply(extract_ref_and_id))
df_brands.drop('cpg', axis=1, inplace=True)
df_brands

,brand_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_ref,cpg_id
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


### Checking number of Missing values

In [8]:
print(df_brands.shape)
df_brands.isnull().sum()

(1167, 9)


brand_id          0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
cpg_ref           0
cpg_id            0
dtype: int64

In [9]:
# !pip install dataqualityreport
from dataqualityreport import dqr_table
dqr_table(df_brands)

Building summary df...
Constructing box plots...
Spreading hist plots...


,Type,Unique (Cardinality = Row Count)'>Card*Unique,% MissingHeatmap,%Missing,%Zeros,%Negative,Box Plot,Robust Histogram
Column,,,,,,,,
brand_id,O,1k*,,,,,,
barcode,I,1k,,,,,,
category,O,23,,,,,,
categoryCode,O,14,,,,,,
name,O,1k,,,,,,
topBrand,F,2,,,,,,
brandCode,O,897,,,,,,
cpg_ref,O,2,,,,,,
cpg_id,O,196,,,,,,


### Checking column 'topBrand', since it has large amount of Zeros and missing values accroding to above summary

In [10]:
df_brands['topBrand'].value_counts(dropna=False)

NaN    612
0.0    524
1.0     31
Name: topBrand, dtype: int64

We can see, the more than 50% values in 'topBrand' column is missing

### Checking column 'categoryCode', since it has missing values accroding to above summary

In [11]:
df_brands['categoryCode'].value_counts(dropna=False)

NaN                              650
BAKING                           359
CANDY_AND_SWEETS                  71
BEER_WINE_SPIRITS                 31
HEALTHY_AND_WELLNESS              14
GROCERY                           11
BABY                               7
CLEANING_AND_HOME_IMPROVEMENT      6
BREAD_AND_BAKERY                   5
DAIRY_AND_REFRIGERATED             5
PERSONAL_CARE                      4
BEVERAGES                          1
OUTDOOR                            1
MAGAZINES                          1
FROZEN                             1
Name: categoryCode, dtype: int64

We can see that more than 50% values in 'topBrand' column is missing, and most of them are in BAKING category

### Checking redundant/duplicate rows

In [12]:
redundant_records = df_brands.duplicated()

# Display the redundant records
print(df_brands[redundant_records])

Empty DataFrame
Columns: [brand_id, barcode, category, categoryCode, name, topBrand, brandCode, cpg_ref, cpg_id]
Index: []


# Overall , we can summarize, data quality is fine for brands table.

### 4 columns have missing values with 'topBrand' and 'categoryCode' have more than 50% missing values.

### Foreign Key check

In [13]:
df_reward_receipts = pd.read_json('df_reward_receipts.json',lines=True)

In [14]:
df_reward_receipts['barcode'].isin(df_brands['barcode']).any()

False

In [15]:
df_reward_receipts['brandCode'].isin(df_brands['brandCode']).any()

True

We can prove that brandCode acts as a foreign key to rewardReceipt table

In [18]:
# Store the df_brands DataFrame in a JSON file
df_brands.to_json('new_data/df_brands.json', orient='records', lines=True)